# NLST clinical data pipeline
Parse NLST data dictionaries to get labels and biomarkers

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import os
import glob
from os import path
import numpy as np
import matplotlib.pyplot as plt
from math import isnan

### Preprocess step (Step 1)
split into jobs

In [74]:
import glob
done = '/media/litz/Elements/litz/data/nlst/DeepLungScreening/prep'
doneids = [os.path.basename(i).split('_')[0] for i in glob.glob(os.path.join(done, "*.nii.gz"))]
dst = '/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/nlst/jobs'
train_ids = nlst_train[~nlst_train['id'].isin(doneids)]
train_ids = train_ids[['id']]
njobs = 7
job_size = len(train_ids)//njobs
print(len(train_ids), job_size)
for job in range(njobs):
    if job != njobs-1:
        startrange, endrange = job*job_size, (job+1)*job_size
    else:
        startrange, endrange = job*job_size, len(train_ids)
    
    jobids = train_ids.iloc[startrange: endrange]
    dstpath = os.path.join(dst, f"nlst_train_job{job}.csv")
    jobids.to_csv(dstpath, index=False)



1752 250


In [75]:
prep_dir = os.listdir('/media/litz/Elements/litz/data/nlst/DeepLungScreening/prep')
missing = []
for i, row in nlst_train_prep.iterrows():
    fname = f"{row['pid']}time{row['year']}"
    a, b, c = f"{fname}_clean.nii.gz", f"{fname}_clean.npy", f"{fname}_label.npy"
    if a not in prep_dir:
        missing.append(a)
    if b not in prep_dir:
        missing.append(b)
    if c not in prep_dir:
        missing.append(c)
missing



[]

### Preprocessed scans (Step 2, 3, 4)

In [3]:
import sys
sys.path.append("/home/local/VANDERBILT/litz/github/MASILab/cohorts")
from nlst.nlst import NLSTCohort
cohort = NLSTCohort(cache='/home/local/VANDERBILT/litz/github/MASILab/cohorts/nlst/nlst.csv', scan_cache='/home/local/VANDERBILT/litz/github/MASILab/cohorts/nlst/nlst_scan.csv')
scan_cohort = cohort.scan_cohort_df
scan_cohort = scan_cohort[scan_cohort['scandate'].notnull()]
scan_cohort['id'] =scan_cohort['pid'] + 'time' + pd.to_datetime(scan_cohort['scandate']).apply(lambda x: x.year).astype(str)
df = cohort.cohort_df


/home/local/VANDERBILT/litz/github/MASILab/toolkit/datasets/cohort.py:51: DtypeWarning: Columns (99,248,249) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fpath, dtype=dtype)


In [6]:
df['lung_cancer'].value_counts()

0    25633
1     1089
Name: lung_cancer, dtype: int64

In [100]:
import sys
sys.path.append("/home/local/VANDERBILT/litz/github/MASILab/cohorts")
from nlst.nlst import NLSTCohort
# all preprocessed scans from NLST 
scan_dir = "/media/litz/Elements/litz/data/nlst/DeepLungScreening/prep"
rows = []
for scan in glob.glob(os.path.join(scan_dir, "*_clean.nii.gz")):
    scanid = os.path.basename(scan).split("_clean.nii.gz")[0]
    pid, year = scanid.split("time")
    rows.append({"pid": pid, "year": year, "id": scanid})
scan_df = pd.DataFrame(rows)

# scan cohort
cohort = NLSTCohort(cache='/home/local/VANDERBILT/litz/github/MASILab/cohorts/nlst/nlst.csv', scan_cache='/home/local/VANDERBILT/litz/github/MASILab/cohorts/nlst/nlst_scan.csv')
scan_cohort = cohort.scan_cohort_df
scan_cohort = scan_cohort[scan_cohort['scandate'].notnull()]
scan_cohort['id'] =scan_cohort['pid'] + 'time' + pd.to_datetime(scan_cohort['scandate']).apply(lambda x: x.year).astype(str)

# ardila test set
ardila = pd.read_excel('/home/local/VANDERBILT/litz/github/MASILab/cohorts/nlst/ardila_test_set.xlsx', dtype={'patient_id':str})
# scan_cohort['scandate'] = scan_cohort['scandate'].astype(int)
scan_cohort['study_yr'] = (scan_cohort.groupby('pid')['scandate'].rank(method='dense', ascending=True) - 1).astype(int)
test_cohort = scan_cohort.merge(ardila, left_on=['pid', 'study_yr'], right_on=['patient_id', 'study_yr'], how='inner')
test_cohort = test_cohort.groupby(['pid', 'study_yr'], as_index=False).max()

# nodule training cohort, removing subjects without nodules, test set, and scans without preprocessing
scan_prep = scan_cohort.merge(scan_df, on=['pid', 'id'])
print(f"All scans = {len(scan_cohort)} | Preproc scans = {len(scan_prep)}")
scan_prep.to_csv('/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/nlst/nlst_cohort_prep_v4.csv', index=False)
# scan_cohort.to_csv('/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/nlst/nlst_cohort_all.csv')

/home/local/VANDERBILT/litz/github/MASILab/toolkit/datasets/cohort.py:52: DtypeWarning: Columns (99,248,249) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fpath, dtype=dtype)


All scans = 71332 | Preproc scans = 69249


### From raw data

In [34]:
# patient dictionaries
root_dir = "/nfs/masi/NLST/package-nlst-7-2018.09.24"
raw_cli = path.join(root_dir, "participant.data.d100517.csv")
raw_df = pd.read_csv(raw_cli, dtype={"pid": str})
raw_df

/tmp/ipykernel_2593076/2716169.py:4: DtypeWarning: Columns (99,248,249) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(raw_cli, dtype={"pid": str})


,cen,dataset_version,elig,ineligible,pid,rndgroup,study,age,educat,ethnic,...,progsite_pleura_ever,progsite_pleura_num,progsite_skin_1st,progsite_skin_days,progsite_skin_ever,progsite_skin_num,progsite_unk_1st,progsite_unk_days,progsite_unk_ever,progsite_unk_num
0,BG,2011.02.03/10.05.17,2,NaN,100001,2,1,70,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AF,2011.02.03/10.05.17,2,NaN,100002,1,1,66,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AR,2011.02.03/10.05.17,2,NaN,100003,2,1,64,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AF,2011.02.03/10.05.17,2,NaN,100004,1,1,60,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AA,2011.02.03/10.05.17,2,NaN,100005,1,1,64,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53447,AE,2011.02.03/10.05.17,2,NaN,218890,1,2,73,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53448,BA,2011.02.03/10.05.17,2,NaN,218891,2,3,66,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53449,BF,2011.02.03/10.05.17,2,NaN,218892,1,2,56,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53450,AJ,2011.02.03/10.05.17,2,NaN,218893,1,3,69,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Find images that pass QA or preprocessing
1. First step get all images from the nfs directory
2. After preprocessing, revise cohort with images that were able to be preprocessed

In [3]:
# QA'd imaging
qa_dirs = [
    "/nfs/masi/NLST/nifti/NIFTI_cancer/", "/nfs/masi/NLST/nifti/NIFTI_nocancer/",
    "/nfs/masi/NLST/nifti/NIFTI_notobtain", "/nfs/masi/NLST/nifti/NIFTI_pending"
]
rows = []
# recursively get all files in all subdirectories
for qa_dir in qa_dirs:  
    for pid in os.listdir(qa_dir):
        for year in os.listdir(os.path.join(qa_dir, pid)):
            for filename in os.listdir(os.path.join(qa_dir, pid, year)):
                if "time" in filename:
                    rows.append({"pid":pid, "year": year, "id": filename.split(".nii.gz")[0]})

# print(scans[:10])
print(len(rows))

KeyboardInterrupt: 

In [3]:
# images that passed preprocessing
scan_dir = "/home/local/VANDERBILT/litz/data/nlst/DeepLungScreening/prep"
rows = []
for scan in glob.glob(os.path.join(scan_dir, "*_clean.nii.gz")):
    scanid = os.path.basename(scan).split("_clean.nii.gz")[0]
    pid, year = scanid.split("time")
    rows.append({"pid": pid, "year": year, "id": scanid})


In [4]:
scan_df = pd.DataFrame(rows)
scan_df = scan_df.sort_values(by=['pid', 'id'])
scan_df['session'] = scan_df.groupby(['pid'])['year'].rank('dense', ascending=True) # assign a T0, T1, or T2 for each scan
scan_df['session'] = scan_df['session'] - 1
scan_df['session'] = scan_df['session'].astype(int)

In [5]:
scan_df

,pid,year,id,session
13154,100004,1999,100004time1999,0
13245,100004,2000,100004time2000,1
5045,100004,2001,100004time2001,2
772,100012,1999,100012time1999,0
876,100012,2000,100012time2000,1
...,...,...,...,...
10414,218819,1999,218819time1999,0
808,218819,2000,218819time2000,1
8298,218819,2001,218819time2001,2
7533,218866,1999,218866time1999,0


## Lung screening imaging cohort
Riqiang/PLCO biomarkers: age, education ,bmi, personal cancer history, family lung cancer history, tobacco use, tobacco use quit time, pack years

**Gao R, Tang Y, Khan MS, Xu K, Paulson AB, Sullivan S, Huo Y, Deppen S, Massion PP, Sandler KL, Landman BA. Cancer Risk Estimation Combining Lung Screening CT with Clinical Data Elements. Radiol Artif Intell. 2021 Oct 13;3(6):e210032. doi: 10.1148/ryai.2021210032. PMID: 34870220; PMCID: PMC8637232.**

In [6]:
# extract desired features - matching 
demo_ft = ['pid', 'age', 'gender', 'educat', 'race', 'ethnic', 'height', 'weight','rndgroup']
copd_ft = ['diagcopd']
pmh_ft = ['cancblad', 'cancbrea', 'canccerv', 'canccolo', 'cancesop', 'canckidn', 'canclary',
    'cancnasa', 'cancoral', 'cancpanc', 'cancphar', 'cancstom', 'cancthyr', 'canctran'] # past medical history features
plc_ft = ['canclung']
fmh_ft = ['fambrother', 'famchild', 'famfather', 'fammother', 'famsister'] # family history of lung cancer
smoking_ft = ['cigsmok', 'age_quit', 'pkyr', 'smokeage', 'smokeday']
lc_ft = ['can_scr', 'conflc', 'cancyr', 'candx_days'] # biopsy-confirmed lung cancer
sc_ft = demo_ft + copd_ft+pmh_ft + plc_ft+fmh_ft+smoking_ft+lc_ft
sc_df = raw_df[sc_ft]

### apply inclusion/exclusion criteria
![inclusion criteria!](./screening_cohort_criteria.png)

In [7]:
# apply inclusion/exclusion criteria
sc_df = sc_df[sc_df['rndgroup']==1] # CT arm
sc_df = sc_df[(sc_df['age']>=55) & (sc_df['age'] <= 74)] # age
sc_df = sc_df[sc_df['pkyr'] >=30] # pkr
sc_df['canclung'] = sc_df['canclung'].fillna(0) # prior lung cancer
sc_df = sc_df[sc_df['canclung']==0]
# assume NLST excluded subjects w/ chest ct within 18 mo, hemoptysis, weight loss



### Preprocess and derive features
- BMI
- PMH and FH collapse into single features
- tobacco use status: current or former smoker?
- smoking quit time: calc yrs bw smoking cessation age and screening age
- pack-years

In [9]:
# demo
sc_df['age'] = sc_df['age']
def parse_edu(x):
    mapper = {
        1:1, # less than high school
        2:1,
        3:2, # high school
        4:3, # post high school training
        5:4, # some college
        6:5, # college degree
        7:6, # graduate degree
        8:0, # other, missing, decline to answer
        95:0,
        99:0,
        98:0,
    }
    return mapper[x]    
sc_df['educat'] = sc_df['educat'].apply(lambda x: parse_edu(x))
def parse_race(x):
    mapper= {
        1:1, #White
        2:2, #Black
        3:4, #Asian
        4:5, #American Indian or Alaskan Native
        5:6, #Native Hawaiian or Other Pacific Islander
        6:0, #Mixed, other, missing, unkown, decline to answer
        7:0,
        95:0,
        96:0,
        98:0,
        99:0
    }
    return mapper[x]
sc_df['race'] = sc_df['race'].apply(lambda x: parse_race(x))

# set race to 3 if ethnicity is hispanic
def parse_ethnic(x):
    mapper = {
        1:1, # hispanic
        2:0, # neither hispanic nor latino, missing, decline to answer
        7:0,
        95:0,
        98:0,
        99:0
    }
    return mapper[x]
sc_df['ethnic'] = sc_df['ethnic'].apply(lambda x: parse_ethnic(x))
sc_df.loc[sc_df['ethnic']==1, 'race'] = 3

# calculate bmi = kg/m^2
sc_df['weight'] = 0.45359237*sc_df['weight'] # lb to kg
sc_df['height'] = 0.0254*sc_df['height'] # in to m
sc_df['bmi'] = sc_df['weight'].div(np.power(sc_df['height'], 2))
sc_df['race'].value_counts(dropna=False)

1    23933
2     1184
4      546
3      479
0      387
6       87
5       82
Name: race, dtype: int64

### COPD and emphysema

In [10]:
# copd
sc_df['copd'] = sc_df['diagcopd']
print(sc_df['copd'].value_counts(dropna=False))

# emphysema as a reported abnormality from radiology read
abnorm_df = pd.read_csv("/nfs/masi/NLST/package-nlst-7-2018.09.24/Spiral CT Abnormalities/sct_abnormalities.data.d100517.csv", dtype={'pid':str})
abnorm_df = abnorm_df[abnorm_df['sct_ab_desc']==59]
abnorm_df = abnorm_df.groupby('pid', as_index=False)['sct_ab_desc'].max()
sc_df = sc_df.merge(abnorm_df, on=['pid'], how='left')
sc_df['emphysema']=~sc_df['sct_ab_desc'].isnull()
sc_df['emphysema'].value_counts(dropna=False)

0.0    25216
1.0     1343
NaN      139
Name: copd, dtype: int64


False    15096
True     11602
Name: emphysema, dtype: int64

In [11]:
# aggregate PMH and FH to single features

# fill na and convert to intergers
sc_df[pmh_ft + fmh_ft] = sc_df[pmh_ft + fmh_ft].fillna(0)
sc_df[pmh_ft + fmh_ft] = sc_df[pmh_ft + fmh_ft].astype(bool) 

# pmh true if any pmh feature true
sc_df['phist'] = sc_df[pmh_ft].isin([1]).any(axis=1)

# fh true if any fh feature true
sc_df['fhist'] = sc_df[fmh_ft].isin([1]).any(axis=1)

print(f"PMH: \n {sc_df['phist'].value_counts()}")
print(f"FH: \n {sc_df['fhist'].value_counts()}")


PMH: 
 False    25618
True      1080
Name: phist, dtype: int64
FH: 
 False    20889
True      5809
Name: fhist, dtype: int64


In [12]:
# smoking quit time: calc yrs bw smoking cessation age and screening age
sc_df = sc_df[sc_df['cigsmok'].notnull()] # drop subjects with a NaN smoking status
sc_df = sc_df[sc_df['smokeage'].notnull()] # drop subjects with a NaN smoking age
sc_df['cigsmok'] = sc_df['cigsmok'].astype(int)
sc_df['quit_time'] = sc_df['age'] - sc_df['age_quit']
sc_df['quit_time'] = sc_df['quit_time'].apply(lambda x: 0 if x < 0 else x).fillna(0)
sc_df['quit_time'].unique()

# smoking duration: number years smoked
def get_smo_duration(x):
    if not isnan(x['age_quit']):
        delta = x['age_quit'] - x['smokeage']
    else:
        delta = x['age'] - x['smokeage']
    return max(delta, 0)
sc_df['smo_duration'] = sc_df.apply(lambda x: get_smo_duration(x), axis=1)

# smoking intensity: average number of cig smoked per day
sc_df['smo_intensity'] = sc_df['smokeday']


### Positive/Negative labels for patients with scans
positive = biopsy-confirmed diagnosis of lung cancer within 2 years of imaging date

In [13]:
# Identify Cases/Controls
pos = sc_df[sc_df['can_scr']!=0] # lung cancer report
pos = pos[pos['cancyr']<5]  # study year associated with cancer. drop year 5 and greater
pos['lung_cancer'] = 1 
pos = pos[['pid', 'lung_cancer']]
sc_df = sc_df.merge(pos, on=['pid'], how='left')
sc_df['lung_cancer'] = sc_df['lung_cancer'].fillna(0)
sc_df['lung_cancer'].value_counts()


0.0    25806
1.0      891
Name: lung_cancer, dtype: int64

In [14]:
merged = sc_df.merge(scan_df, on='pid', how='inner')
merged

,pid,age,gender,educat,race,ethnic,height,weight,rndgroup,diagcopd,...,emphysema,phist,fhist,quit_time,smo_duration,smo_intensity,lung_cancer,year,id,session
0,100004,60,1,4,1,0,1.7780,92.986436,1,0.0,...,False,False,False,15.0,23.0,40,0.0,1999,100004time1999,0
1,100004,60,1,4,1,0,1.7780,92.986436,1,0.0,...,False,False,False,15.0,23.0,40,0.0,2000,100004time2000,1
2,100004,60,1,4,1,0,1.7780,92.986436,1,0.0,...,False,False,False,15.0,23.0,40,0.0,2001,100004time2001,2
3,100012,61,2,6,1,0,1.7018,64.410117,1,0.0,...,False,False,False,0.0,39.0,20,1.0,1999,100012time1999,0
4,100012,61,2,6,1,0,1.7018,64.410117,1,0.0,...,False,False,False,0.0,39.0,20,1.0,2000,100012time2000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16853,218819,66,2,3,1,0,1.7272,58.967008,1,0.0,...,True,False,False,0.0,52.0,20,0.0,1999,218819time1999,0
16854,218819,66,2,3,1,0,1.7272,58.967008,1,0.0,...,True,False,False,0.0,52.0,20,0.0,2000,218819time2000,1
16855,218819,66,2,3,1,0,1.7272,58.967008,1,0.0,...,True,False,False,0.0,52.0,20,0.0,2001,218819time2001,2
16856,218866,57,1,6,1,0,1.8288,77.110703,1,0.0,...,False,False,True,4.0,38.0,18,0.0,1999,218866time1999,0


In [15]:

ft = ['pid', 'id', 'session', 'with_image', 'with_marker', 'age', 'gender', 'race', 'education',  'bmi',  'copd', 'emphysema', 'phist', 'fhist', 
    'smo_status', 'smo_duration', 'smo_intensity', 'quit_time', 'pkyr', 'lung_cancer']
merged['education'] = merged['educat']
merged['smo_status'] = merged['cigsmok']
merged['with_image'] = True
merged['with_marker'] = True
cohort_df = merged[ft]
cohort_df

,pid,id,session,with_image,with_marker,age,gender,race,education,bmi,copd,emphysema,phist,fhist,smo_status,smo_duration,smo_intensity,quit_time,pkyr,lung_cancer
0,100004,100004time1999,0,True,True,60,1,1,4,29.414135,0.0,False,False,False,0,23.0,40,15.0,34.0,0.0
1,100004,100004time2000,1,True,True,60,1,1,4,29.414135,0.0,False,False,False,0,23.0,40,15.0,34.0,0.0
2,100004,100004time2001,2,True,True,60,1,1,4,29.414135,0.0,False,False,False,0,23.0,40,15.0,34.0,0.0
3,100012,100012time1999,0,True,True,61,2,1,6,22.240116,0.0,False,False,False,1,39.0,20,0.0,37.0,1.0
4,100012,100012time2000,1,True,True,61,2,1,6,22.240116,0.0,False,False,False,1,39.0,20,0.0,37.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16853,218819,218819time1999,0,True,True,66,2,1,3,19.766230,0.0,True,False,False,1,52.0,20,0.0,52.0,0.0
16854,218819,218819time2000,1,True,True,66,2,1,3,19.766230,0.0,True,False,False,1,52.0,20,0.0,52.0,0.0
16855,218819,218819time2001,2,True,True,66,2,1,3,19.766230,0.0,True,False,False,1,52.0,20,0.0,52.0,0.0
16856,218866,218866time1999,0,True,True,57,1,1,6,23.055908,0.0,False,False,True,0,38.0,18,4.0,33.3,0.0


In [16]:
cohort_df.groupby('pid').max()

,id,session,with_image,with_marker,age,gender,race,education,bmi,copd,emphysema,phist,fhist,smo_status,smo_duration,smo_intensity,quit_time,pkyr,lung_cancer
pid,,,,,,,,,,,,,,,,,,,
100004,100004time2001,2,True,True,60,1,1,4,29.414135,0.0,False,False,False,0,23.0,40,15.0,34.0,0.0
100012,100012time2000,1,True,True,61,2,1,6,22.240116,0.0,False,False,False,1,39.0,20,0.0,37.0,1.0
100019,100019time2001,2,True,True,61,1,1,4,23.962608,0.0,True,False,False,0,43.0,40,3.0,78.0,0.0
100026,100026time2001,2,True,True,57,1,1,3,35.146505,0.0,False,False,False,0,41.0,30,0.0,61.5,0.0
100035,100035time2001,2,True,True,55,2,1,3,22.096473,0.0,True,False,False,1,38.0,20,0.0,38.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218705,218705time2001,2,True,True,68,1,1,4,37.305733,0.0,True,False,False,1,43.0,40,0.0,84.0,0.0
218750,218750time2001,2,True,True,55,2,1,5,19.485425,0.0,True,False,True,1,35.0,20,0.0,35.0,0.0
218762,218762time2001,2,True,True,55,2,1,5,20.967282,0.0,True,False,False,1,41.0,20,0.0,41.0,1.0


In [58]:
cohort_df['phist'].value_counts()

False    16068
True       790
Name: phist, dtype: int64

In [55]:
out_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/nlst_cohort_prep_v2.csv"
cohort_df.to_csv(out_path, index_label=False)
a = pd.read_csv(out_path)

In [59]:
a = cohort_df.groupby('pid').max()
len(a)

6230

### Lung Screening Multimodal cohort

In [133]:
# extract desired features - matching 
demo_ft = ['pid', 'age', 'educat', 'race', 'ethnic', 'height', 'weight','rndgroup']
copd_ft = ['diagcopd']
pmh_ft = ['cancblad', 'cancbrea', 'canccerv', 'canccolo', 'cancesop', 'canckidn', 'canclary',
    'cancnasa', 'cancoral', 'cancpanc', 'cancphar', 'cancstom', 'cancthyr', 'canctran'] # past medical history features
plc_ft = ['canclung']
fmh_ft = ['fambrother', 'famchild', 'famfather', 'fammother', 'famsister'] # family history of lung cancer
smoking_ft = ['cigsmok', 'age_quit', 'pkyr']
lc_ft = ['can_scr', 'conflc', 'cancyr', 'candx_days'] # biopsy-confirmed lung cancer
sc_ft = demo_ft + copd_ft+pmh_ft + plc_ft+fmh_ft+smoking_ft+lc_ft
sc_df = raw_df[sc_ft]

# apply inclusion/exclusion criteria
sc_df = sc_df[sc_df['rndgroup']==1] # CT arm
sc_df = sc_df[(sc_df['age']>=55) & (sc_df['age'] <= 74)] # age
sc_df = sc_df[sc_df['pkyr'] >=30] # pkr
sc_df['canclung'] = sc_df['canclung'].fillna(0) # prior lung cancer
sc_df = sc_df[sc_df['canclung']==0]

# demo
sc_df['age'] = sc_df['age']
def parse_edu(x):
    mapper = {
        1:1, # less than high school
        2:1,
        3:2, # high school
        4:3, # post high school training
        5:4, # some college
        6:5, # college degree
        7:6, # graduate degree
        8:0, # other, missing, decline to answer
        95:0,
        99:0,
        98:0,
    }
    return mapper[x]    
sc_df['educat'] = sc_df['educat'].apply(lambda x: parse_edu(x))
def parse_race(x):
    mapper= {
        1:1, #White
        2:2, #Black
        3:4, #Asian
        4:5, #American Indian or Alaskan Native
        5:6, #Native Hawaiian or Other Pacific Islander
        6:0, #Mixed, other, missing, unkown, decline to answer
        7:0,
        95:0,
        96:0,
        98:0,
        99:0
    }
    return mapper[x]
sc_df['race'] = sc_df['race'].apply(lambda x: parse_race(x))

# set race to 3 if ethnicity is hispanic
def parse_ethnic(x):
    mapper = {
        1:1, # hispanic
        2:0, # neither hispanic nor latino, missing, decline to answer
        7:0,
        95:0,
        98:0,
        99:0
    }
    return mapper[x]
sc_df['ethnic'] = sc_df['ethnic'].apply(lambda x: parse_ethnic(x))
sc_df.loc[sc_df['ethnic']==1, 'race'] = 3

# calculate bmi = kg/m^2
sc_df['weight'] = 0.45359237*sc_df['weight'] # lb to kg
sc_df['height'] = 0.0254*sc_df['height'] # in to m
sc_df['bmi'] = sc_df['weight'].div(np.power(sc_df['height'], 2))
sc_df['race'].value_counts(dropna=False)

# copd
sc_df['copd'] = sc_df['diagcopd']

# emphysema as a reported abnormality from radiology read
abnorm_df = pd.read_csv("/nfs/masi/NLST/package-nlst-7-2018.09.24/Spiral CT Abnormalities/sct_abnormalities.data.d100517.csv", dtype={'pid':str})
abnorm_df = abnorm_df[abnorm_df['sct_ab_desc']==59]
abnorm_df = abnorm_df.groupby('pid', as_index=False)['sct_ab_desc'].max()
sc_df = sc_df.merge(abnorm_df, on=['pid'], how='left')
sc_df['emphysema']=~sc_df['sct_ab_desc'].isnull()
sc_df['emphysema'].value_counts(dropna=False)

# aggregate PMH and FH to single features

# fill na and convert to intergers
sc_df[pmh_ft + fmh_ft] = sc_df[pmh_ft + fmh_ft].fillna(0)
sc_df[pmh_ft + fmh_ft] = sc_df[pmh_ft + fmh_ft].astype(bool) 

# pmh true if any pmh feature true
sc_df['phist'] = sc_df[pmh_ft].isin([1]).any(axis=1)

# fh true if any fh feature true
sc_df['fhist'] = sc_df[fmh_ft].isin([1]).any(axis=1)

# smoking quit time: calc yrs bw smoking cessation age and screening age
sc_df = sc_df[sc_df['cigsmok'].notnull()] # drop subjects with a NaN smoking status
sc_df['cigsmok'] = sc_df['cigsmok'].astype(int)
sc_df['quit_time'] = sc_df['age'] - sc_df['age_quit']
sc_df['quit_time'] = sc_df['quit_time'].apply(lambda x: 0 if x < 0 else x).fillna(0)


In [134]:
# Identify Cases/Controls
pos = sc_df[sc_df['can_scr']!=0] # lung cancer report
pos = pos[pos['cancyr']<5]  # study year associated with cancer. drop year 5 and greater
pos['lung_cancer'] = 1 
pos = pos[['pid', 'lung_cancer']]
sc_df = sc_df.merge(pos, on=['pid'], how='left')
sc_df['lung_cancer'] = sc_df['lung_cancer'].fillna(0)
sc_df['lung_cancer'].value_counts()


0.0    25807
1.0      891
Name: lung_cancer, dtype: int64

In [135]:
has_img = scan_df.groupby('pid', as_index=False)[['id', 'session']].max()
merged = sc_df.merge(has_img, on='pid', how='left')
merged['with_image'] = ~pd.isnull(merged['session'])
merged['with_image'].value_counts(dropna=False)

False    20468
True      6230
Name: with_image, dtype: int64

In [136]:
# get final features and rename for DeepLungScreening pipeline
ft = ['pid', 'id', 'session', 'with_image', 'with_marker', 'age', 'race', 'education',  'bmi',  'copd', 'emphysema', 'phist', 'fhist', 'smo_status', 'quit_time', 'pkyr', 'lung_cancer']
merged['education'] = merged['educat']
merged['smo_status'] = merged['cigsmok']
merged['with_marker'] = True
cohort_df = merged[ft]
cohort_df

,pid,id,session,with_image,with_marker,age,race,education,bmi,copd,emphysema,phist,fhist,smo_status,quit_time,pkyr,lung_cancer
0,100002,NaN,NaN,False,True,66,1,2,26.608386,0.0,False,False,False,1,0.0,52.00,0.0
1,100004,100004time2001,2.0,True,True,60,1,4,29.414135,0.0,False,False,False,0,15.0,34.00,0.0
2,100005,NaN,NaN,False,True,64,1,1,34.456518,0.0,True,False,False,0,3.0,92.00,0.0
3,100009,NaN,NaN,False,True,55,1,6,37.122590,0.0,False,False,False,1,0.0,52.50,0.0
4,100010,NaN,NaN,False,True,68,1,4,30.409584,0.0,False,False,False,0,14.0,84.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26693,218888,NaN,NaN,False,True,58,1,4,22.807188,1.0,False,False,False,1,0.0,39.00,0.0
26694,218890,NaN,NaN,False,True,73,1,2,35.359456,0.0,True,False,False,1,0.0,120.00,0.0
26695,218892,NaN,NaN,False,True,56,1,4,23.327154,0.0,True,False,False,0,0.0,38.00,0.0
26696,218893,NaN,NaN,False,True,69,1,3,31.930063,1.0,True,False,False,0,1.0,41.25,0.0


In [137]:
out_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/nlst_cohort_v3.csv"
cohort_df.to_csv(out_path, index_label=False)